读取clinvar数据

In [1]:
import pandas as pd

file_path = r'D:\data\clinvar\variant_summary.txt\variant_summary.txt'
df = pd.read_csv(file_path, delimiter='\t')

ModuleNotFoundError: No module named 'pandas'

In [ ]:
df.columns

In [ ]:
df['ClinicalSignificance'].value_counts()[:10]

In [ ]:
pathogenicity_levels = ['Benign', 'Benign/Likely benign', 'Likely benign', 'Likely pathogenic', 'Pathogenic']
variants_by_pathogenicity = []
for patho in pathogenicity_levels:
    variants_by_pathogenicity.append(df['ClinicalSignificance'].value_counts()[patho])
variants_by_pathogenicity

In [ ]:
df['Chromosome'].value_counts()

In [ ]:
chrs = [str(i) for i in range(1, 23)]
variants_chr_counts = df['Chromosome'].value_counts()
chrs.extend(['X', 'Y', 'MT'])
variants_by_chr = []
for chr in chrs:
    print(chr)
    variants_by_chr.append(variants_chr_counts[chr])
variants_by_chr

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 3))
plt.subplot(121)
plt.bar(pathogenicity_levels, variants_by_pathogenicity)
plt.subplot(122)
plt.bar(chrs, variants_by_chr)
plt.savefig('../img/data_desc/raw_data_description.png')
plt.show()

In [11]:
df['Type'].value_counts()

NameError: name 'df' is not defined

In [2]:
import pandas as pd

file_path = r'D:\data\clinvar\variant_summary.txt\variant_summary.txt'
df = pd.read_csv(file_path, delimiter='\t')
df = df[df['Type']=='single nucleotide variant']
df = df[(df['ClinicalSignificance'] == 'Pathogenic') | (df['ClinicalSignificance'] == 'Benign')]
df = df[df['Assembly'] == 'GRCh38']
df = df[df['Chromosome']!='MT']
df['label'] = -1
df['label'][df['ClinicalSignificance'] == 'Pathogenic'] = 1
df['label'][df['ClinicalSignificance'] != 'Pathogenic'] = 0
df = df[df['ReferenceAlleleVCF'] != 'na']
df.to_csv('../data/variants.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\data\\clinvar\\variant_summary.txt\\variant_summary.txt'

获取突变位点周围序列

In [3]:
import pandas as pd

# 取突变位点周围50bp的内容
surrounding_lens = [50, 100, 300, 500, 1000]
df = pd.read_csv('../data/variants.csv')
for chr, subset in df.groupby('Chromosome'):
    print(chr)
    with open(r'../data/ucsc/hg38/chr{}.txt'.format(chr)) as f:
        seq = f.readline()
        for index, variant in subset.iterrows():
            point_index = int(variant['PositionVCF'])
            # point_index = seq[point_index-1]
            ref = variant['ReferenceAlleleVCF']
            alt = variant['AlternateAlleleVCF']
            for surrounding_len in surrounding_lens:
                left = seq[point_index-1-surrounding_len:point_index-1]
                right = seq[point_index: point_index+surrounding_len]
                # print(point_index, ref, alt, seq[point_index-1], left, right)
                if len(left)<surrounding_len:
                    left = 'N' *(surrounding_len-len(left)) + left
                if len(right)<surrounding_len:
                    right = right + 'N'*(surrounding_len-len(right))
                t1 = ''.join([left, ref, right])
                t2 = ''.join([left, alt, right])
                ref_col = 'seq_{}_ref'.format(surrounding_len)
                alt_col = 'seq_{}_alt'.format(surrounding_len)
                df.loc[index, ref_col] = t1.upper()
                df.loc[index, alt_col] = t1.upper()

1
10
11
12
13
14
15
16
17
18
19
2


In [ ]:
df = df[df['ReferenceAlleleVCF'] != 'na']

In [ ]:
df.to_csv('../data/variants_with_seq.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('../data/variants_with_seq.csv')

In [ ]:
pathogenicity_levels = ['Benign', 'Pathogenic']
variants_by_pathogenicity = []
for patho in pathogenicity_levels:
    print(patho)
    variants_by_pathogenicity.append(df['ClinicalSignificance'].value_counts()[patho])
variants_by_pathogenicity

In [ ]:
chrs = [str(i) for i in range(1, 23)]
variants_chr_counts = df['Chromosome'].value_counts()
chrs.extend(['X', 'Y'])
variants_by_chr = []
for chr in chrs:
    variants_by_chr.append(variants_chr_counts[chr])
variants_by_chr

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 3))
plt.subplot(121)
plt.bar(pathogenicity_levels, variants_by_pathogenicity)
plt.subplot(122)
plt.bar(chrs, variants_by_chr)
plt.savefig('../img/data_desc/fliterd_data_description.png')
plt.show()

In [ ]:
import pandas as pd

df = pd.read_csv('../data/variants_with_seq.csv')

In [ ]:
df_positive = df[df['ClinicalSignificance'] == 'Pathogenic']
df_negative = df[df['ClinicalSignificance'] == 'Benign']

In [ ]:
print(df_positive.shape, df_negative.shape)

In [ ]:
num_p = df_positive.shape[0]

In [ ]:
num_n = df_negative.shape[0]

In [ ]:
train_ratio = 0.7
valid_ration = 0.2
test_ration = 0.1

In [ ]:
df_positive_train = df_positive.iloc[:int(num_p*train_ratio)]
df_positive_valid = df_positive.iloc[int(num_p*train_ratio):int(num_p*0.9)]
df_positive_test = df_positive.iloc[int(num_p*0.9):]

In [ ]:
print(df_positive_train.shape, df_positive_valid.shape, df_positive_test.shape)

In [ ]:
df_negative_train = df_negative.iloc[:int(num_n*train_ratio)]
df_negative_valid = df_negative.iloc[int(num_n*train_ratio):int(num_n*0.9)]
df_negative_test = df_negative.iloc[int(num_n*0.9):]

In [ ]:
print(df_negative_train.shape, df_negative_valid.shape, df_negative_test.shape)

In [ ]:
df_train = pd.concat([df_positive_train, df_negative_train])
df_valid = pd.concat([df_positive_valid, df_negative_valid])
df_test = pd.concat([df_positive_test, df_negative_test])

In [ ]:
print(df_train.shape, df_valid.shape, df_test.shape)

In [ ]:
df_train.to_csv('../data/train.csv', index=False)
df_valid.to_csv('../data/valid.csv', index=False)
df_test.to_csv('../data/test.csv', index=False)